In [ ]:
# Preprocessing Script for RECS 2020 Dataset
# ========================================
# This script loads raw RECS 2020 CSV files, merges them on 'STATE', handles missing values,
# removes outliers, encodes categorical variables, scale features and saves the processed dataset.
#
# Inputs:
# - Raw CSV files in data/raw/
# Outputs:
# - Processed unscaled dataset in data/processed/merged_cleaned.csv for fuzzy logic and app ranges.
# - Processed scaled dataset in data/processed/merged_cleaned_scaled.csv for Decision Tree training.
#
# Dependencies: pandas, numpy, scikit-learn

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Setup paths
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [ ]:
# Load all raw CSV files
consumption = pd.read_csv(os.path.join(RAW_DIR, "recs_annual_household_energy_consumption_and_expenditure.csv"))
housing = pd.read_csv(os.path.join(RAW_DIR, "recs_type_of_housing.csv"))
income = pd.read_csv(os.path.join(RAW_DIR, "recs_income.csv"))
geographic = pd.read_csv(os.path.join(RAW_DIR, "recs_geographic_characteristics.csv"))
year_construction = pd.read_csv(os.path.join(RAW_DIR, "recs_year_of_construction.csv"))
ac_equipment = pd.read_csv(os.path.join(RAW_DIR, "recs_use_of_ac_and_age_of_ac_equipment.csv"))
space_heater = pd.read_csv(os.path.join(RAW_DIR, "recs_use_of_space_heaters_and_age_of_space_heating_equipment.csv"))
water_heater = pd.read_csv(os.path.join(RAW_DIR, "recs_age_of_water_heating_equipment.csv"))

In [ ]:
print(consumption.head())

In [ ]:
print(housing.head())

In [ ]:
print(income.head())

In [ ]:
print(geographic.head())

In [ ]:
print(year_construction.head())

In [ ]:
print(ac_equipment.head())

In [ ]:
print(space_heater.head())

In [ ]:
print(water_heater.head())

In [ ]:
# Drop duplicate 'Total_households' columns (keep consumption's)
for df in [housing, income, year_construction, ac_equipment, space_heater, water_heater]:
    if 'Total_households' in df.columns:
        df.drop(columns=['Total_households'], inplace=True)

# Merge datasets on 'STATE'
merged = consumption.merge(housing, on='STATE', how='inner')
merged = merged.merge(income, on='STATE', how='inner')
merged = merged.merge(geographic, on='STATE', how='inner')
merged = merged.merge(year_construction, on='STATE', how='inner')
merged = merged.merge(ac_equipment, on='STATE', how='inner')
merged = merged.merge(space_heater, on='STATE', how='inner')
merged = merged.merge(water_heater, on='STATE', how='inner')

# Inspect the merged dataset
print("Shape:", merged.shape)
print(merged.head())
print("\nColumns:\n", merged.columns.tolist())

In [ ]:
# Handle missing values with median for numeric columns
numeric_columns = merged.select_dtypes(include=[np.number]).columns
merged[numeric_columns] = merged[numeric_columns].fillna(merged[numeric_columns].median())

In [ ]:
# Remove outliers using IQR method
for col in ['ENERGY_CONSUMPTION_PER_SQFT', 'Pct_INCOME_MORE_THAN_150K', 'Pct_MAIN_HEAT_AGE_OLDER_THAN_20']:
    Q1 = merged[col].quantile(0.25)
    Q3 = merged[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    merged = merged[(merged[col] >= lower_bound) & (merged[col] <= upper_bound)]

In [ ]:
# Ensure CLIMATE_Cold exists
if 'CLIMATE_Cold' not in merged.columns:
    print("Warning: CLIMATE_Cold not found. Creating dummy column with value 0.")
    merged['CLIMATE_Cold'] = 0
else:
    # Encode CLIMATE_Cold as binary (0/1)
    merged['CLIMATE_Cold'] = merged['CLIMATE_Cold'].apply(lambda x: 1 if x in [1, 'Yes', True] else 0)

In [ ]:
# Encode climate variables (ensure binary)
climate_columns = ['CLIMATE_Hot-Humid', 'CLIMATE_Mixed-Humid', 'CLIMATE_Very-Cold']
for col in climate_columns:
    if col in merged.columns:
        merged[col] = merged[col].apply(lambda x: 1 if x in [1, 'Yes', True] else 0)
    else:
        print(f"Warning: {col} not found. Creating dummy column with value 0.")
        merged[col] = 0

In [ ]:
# Select required features
selected_features = [
    'STATE', 'ENERGY_CONSUMPTION_PER_SQFT', 'Pct_INCOME_MORE_THAN_150K',
    'Pct_HOUSING_SINGLE_FAMILY_HOME_DETACHED', 'Pct_HOUSING_APT_MORE_THAN_5_UNITS',
    'CLIMATE_Cold', 'CLIMATE_Hot-Humid', 'CLIMATE_Mixed-Humid', 'CLIMATE_Very-Cold',
    'Pct_BUILT_BEFORE_1950', 'Pct_MAIN_AC_AGE_OLDER_THAN_20',
    'Pct_MAIN_HEAT_AGE_OLDER_THAN_20', 'Pct_MAIN_WATER_HEAT_OLDER_THAN_20'
]
available_features = [col for col in selected_features if col in merged.columns]
merged = merged[available_features]

In [ ]:
# Save the processed dataset
output_path = os.path.join(PROCESSED_DIR, "merged_cleaned.csv")
merged.to_csv(output_path, index=False)

# Print summary
print(f"Processed unscaled dataset saved to {output_path}")
print("Shape:", merged.shape)
print("Columns:", merged.columns.tolist())

In [ ]:
# Scale continuous features
continuous_features = [
    'ENERGY_CONSUMPTION_PER_SQFT', 'Pct_INCOME_MORE_THAN_150K',
    'Pct_HOUSING_SINGLE_FAMILY_HOME_DETACHED', 'Pct_HOUSING_APT_MORE_THAN_5_UNITS',
    'Pct_BUILT_BEFORE_1950', 'Pct_MAIN_AC_AGE_OLDER_THAN_20',
    'Pct_MAIN_HEAT_AGE_OLDER_THAN_20', 'Pct_MAIN_WATER_HEAT_OLDER_THAN_20'
]
continuous_features = [f for f in continuous_features if f in merged.columns]
scaler = StandardScaler()
scaled_data = merged.copy()
scaled_data[continuous_features] = scaler.fit_transform(merged[continuous_features])

# Save scaled dataset
scaled_output_path = os.path.join(PROCESSED_DIR, 'merged_cleaned_scaled.csv')
scaled_data.to_csv(scaled_output_path, index=False)
print(f'Saved scaled dataset: {scaled_output_path}')

# Print summary
print('Shape:', merged.shape)
print('Columns:', merged.columns.tolist())